# Creating N-Gram Language Models

Language models play a crucial role in natural language processing (NLP), powering applications such as text prediction, machine translation, and sentiment analysis. In this project, we explore key concepts behind language modeling, including probabilistic models and text preprocessing techniques.

## Imports

In [5]:
import pandas as pd
import numpy as np
import re
import requests
import time

## Getting  the Text

We will create a function `get_book(url)` that takes in a `url` of a book's "Plain Text UTF-8" from a  public domain book from [Project Gutenberg](https://www.gutenberg.org/), where we will use the `requests` module to download the text and perform text preprocessing to get the book's content.  

In [6]:
def get_book(url):

    # getting text through an HTTP request.
    response = requests.get(url) 

    # delaying 0.5 seconds to follow robot.txt policy.
    time.sleep(0.5)

    # cleaning
    text = response.text.replace('\r\n','\n') 

    # getting start index of book
    start_index = re.search(r"\*\*\* START OF THE PROJECT GUTENBERG EBOOK .*? \*\*\*", text).end() 

    # getting last index of book
    end_index = re.search(r"\*\*\* END OF THE PROJECT GUTENBERG EBOOK .*? \*\*\*", text).start()

    # returning book text
    return text[start_index:end_index]

Extracting text from *Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley* using `get_book(url)`

In [7]:
Frankenstein = get_book("https://www.gutenberg.org/cache/epub/84/pg84.txt")

print(Frankenstein[:5000]) # printing first 5000 characters



Frankenstein;

or, the Modern Prometheus

by Mary Wollstonecraft (Godwin) Shelley


 CONTENTS

 Letter 1
 Letter 2
 Letter 3
 Letter 4
 Chapter 1
 Chapter 2
 Chapter 3
 Chapter 4
 Chapter 5
 Chapter 6
 Chapter 7
 Chapter 8
 Chapter 9
 Chapter 10
 Chapter 11
 Chapter 12
 Chapter 13
 Chapter 14
 Chapter 15
 Chapter 16
 Chapter 17
 Chapter 18
 Chapter 19
 Chapter 20
 Chapter 21
 Chapter 22
 Chapter 23
 Chapter 24




Letter 1

_To Mrs. Saville, England._


St. Petersburgh, Dec. 11th, 17—.


You will rejoice to hear that no disaster has accompanied the
commencement of an enterprise which you have regarded with such evil
forebodings. I arrived here yesterday, and my first task is to assure
my dear sister of my welfare and increasing confidence in the success
of my undertaking.

I am already far north of London, and as I walk in the streets of
Petersburgh, I feel a cold northern breeze play upon my cheeks, which
braces my nerves and fills me with delight. Do you understand this
feeling? Th

## Tokenizing the Text

Tokenization is the process of splitting text into smaller units, which are called tokens. These tokens can be words, subwords, or characters, depending on the level of tokenization used. This is an important step since this is what the model processes.

We will create a function `tokenize` which takes in a string, `book_string`, and returns a **list of the tokens** (words, numbers, and all punctuation) in the book such that:
* The start of every paragraph is represented in the list with the single character `'\x02'`.
* The end of every paragraph is represented in the list with the single character `'\x03'`.
* Tokens include *no* whitespace.
* Two or more newlines count as a paragraph break, and whitespace (e.g. multiple newlines) between two paragraphs of text do not appear as tokens.
* All punctuation marks except `_` count as tokens, even if they are uncommon (e.g. `'@'`, `'+'`, and `'%'` are all valid tokens).

In [8]:
def tokenize(book_string):
    
    # Removing any leading or trailing white spaces
    string = book_string.strip('\x03').strip('\x02').strip() 

    # Replacing paragraph breaks (two or more newlines) with paragraph markers
    new_string = '\x02 ' + re.sub(r'\n{2,}', ' \x03 \x02 ', string) +" \x03"

    # Separating punctuation from words by adding spaces around them, then splitting into list of tokens
    return re.sub(r'[^\w\s]', lambda x: ' '+x.group(0)+' ', new_string).split()

Tokenizing *Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley*

In [9]:
Frankenstein_tokenized = tokenize(Frankenstein)

Frankenstein_tokenized[:50] # first 50 tokens

['\x02',
 'Frankenstein',
 ';',
 '\x03',
 '\x02',
 'or',
 ',',
 'the',
 'Modern',
 'Prometheus',
 '\x03',
 '\x02',
 'by',
 'Mary',
 'Wollstonecraft',
 '(',
 'Godwin',
 ')',
 'Shelley',
 '\x03',
 '\x02',
 'CONTENTS',
 '\x03',
 '\x02',
 'Letter',
 '1',
 'Letter',
 '2',
 'Letter',
 '3',
 'Letter',
 '4',
 'Chapter',
 '1',
 'Chapter',
 '2',
 'Chapter',
 '3',
 'Chapter',
 '4',
 'Chapter',
 '5',
 'Chapter',
 '6',
 'Chapter',
 '7',
 'Chapter',
 '8',
 'Chapter',
 '9']

## Creating a Unigram Language Model

A unigram language model is a model in which the probability assigned to a token is equal to the proportion of tokens in the text(corpus) that are equal to the said token. 

Example:

```py
>>> corpus = 'hello, I am me, when I travel, I become happy, when I am at home, I am content.'
>>> tokenize(corpus)
['\x02', 'hello', ',', 'I', 'am', 'me', ',', 'when', 'I', 'travel', ',', 'I', 'become', 'happy', ',', 'when', 'I', 'am', 'at', 'home', ',', 'I', 'am', 'content', '.', '\x03']
```

Here, there are 26 total tokens. 5 of them are equal to `'I'`, so $P(\text{I}) = \frac{5}{26}$. Here, the Series that the `train` method in our class below should return probability distribution of tokens:

| Token    | Count | Probability |
|----------|-------|-------------|
| `'\x02'` | 1   | $\frac{1}{26}$ |
| `'hello'` | 1   | $\frac{1}{26}$ |
| `','` | 5   | $\frac{5}{26}$ |
| `'I'` | 5   | $\frac{5}{26}$ |
| `'am'` | 3   | $\frac{3}{26}$ |
| `'me'` | 1   | $\frac{1}{26}$ |
| `'when'` | 2   | $\frac{2}{26}$ |
| `'travel'` | 1   | $\frac{1}{26}$ |
| `'become'` | 1   | $\frac{1}{26}$ |
| `'happy'` | 1   | $\frac{1}{26}$ |
| `'at'` | 1   | $\frac{1}{26}$ |
| `'home'` | 1   | $\frac{1}{26}$ |
| `'content'` | 1   | $\frac{1}{26}$ |
| `'.'` | 1   | $\frac{1}{26}$ |
| `'\x03'` | 1   | $\frac{1}{26}$ |

Each probability is calculated as:  

$$P(\text{token}) = \frac{\text{count of token}}{\text{total tokens}}$$  

As before, the `probability` method should take in a tuple and return its probability, using the probabilities stored in `mdl`. For instance, suppose the input tuple is `(',', 'I', 'am', 'content', '.')`. Then,  

$$P(\text{, I am content .}) = P(\text{,}) \cdot P(\text{I}) \cdot P(\text{am}) \cdot P(\text{content}) \cdot P(\text{.})$$  

Substituting the probabilities from our table:  

$$P(\text{, I am content.}) = \frac{5}{26} \cdot \frac{5}{26} \cdot \frac{3}{26} \cdot \frac{1}{26} \cdot \frac{1}{26}$$  

The `sample` method accounts for the fact that not all tokens are equally likely to be sampled.

In [10]:
class UnigramLM(object):
    
    def __init__(self, tokens):

        self.mdl = self.train(tokens)
    
    def train(self, tokens):
        return pd.Series(tokens).value_counts() / len(tokens)
    
    def probability(self, words):
        try:
            if len(words)==1:
                return self.mdl.loc[words]
            else:
                return float((self.mdl.loc[list(words)]).prod())
        except KeyError as e:
            return 0
        
    def sample(self, M):
        return ' '.join(np.random.choice(self.mdl.index,M,p=self.mdl.values))

Below we try our unigram model using the same Frankenstein tokenized text/corpus.

In [11]:
# initializing unigram object with tokenized text that will be trained to probability distribution df(mdl df)
unigram = UnigramLM(Frankenstein_tokenized) 

# probability distribution of frankenstein text
print(unigram.mdl)

print()

# gives probability of input tokenized text
print(f'Expected Probability of "my dear": {unigram.probability(('my','dear'))}') 

print()
# Generates a random sequence of 75 words.
print(unigram.sample(75) )


,                 0.056607
the               0.044651
and               0.033539
.                 0.033345
I                 0.032500
                    ...   
salvation         0.000011
timorous          0.000011
irreproachable    0.000011
indecent          0.000011
thinks            0.000011
Name: count, Length: 7416, dtype: float64

Expected Probability of "my dear": 1.4072164782064333e-05

my , liberty as knowledge which . not accounted like , My Ingolstadt eyes Under sufficiency was , way the happen court league had myself by and ever , long of could but . between unfit tears but little to desirous you scenes your fuel arranged of that . , one the . sudden remain and that having . the duties Well in great Empires_ but the had fill “ At detestation and and I


## Creating an N-gram Language Model

Now, the issue with the previous model is that it does not condition of previous words. It assumes independence between words in a sentence, which is wrong. Now, we will perform conditional probability to make sure sequences of words make sense.


The model would assume that the probability that a token occurs depends only on the previous $N-1$ tokens, rather than all previous tokens. That is:

$$P(w_n|w_1,\ldots,w_{n-1}) = P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

So, in an N-Gram language model, we get to choose, $N$. For any $N$, the resulting N-Gram model looks at the previous $N-1$ tokens when computing probabilities. So, the previous model we made was a 1-Gram model known as a unigram.

<br>

#### Trigram Model Example

When $N=3$, we have a "trigram" model. This model looks at the previous $N-1 = 2$ tokens when computing probabilities if possible.

Consider the tuple `('when', 'I', 'travel', 'I', 'become', 'happy')`. Under the trigram model, the probability of this sentence is computed as follows:

\[
\begin{aligned}
P(\text{when I travel I become happy}) &= P(\text{when}) \cdot P(\text{I | when}) \cdot P(\text{travel | when I}) \cdot \\
&\quad P(\text{I | I travel}) \cdot P(\text{become | travel I}) \cdot P(\text{happy | I become})
\end{aligned}
\]
Note that the first few tokens may not have 2 tokens before it. We will take that into account and go to previous n-gram dataframes when computing these special cases. For example, for the first word, we will go back to the unigram model because it does not have anything before it.



<br>

#### N-Grams

When working with a training text and implementing the `probability` method to compute the probabilities of sequences of words, we work with "chunks" of $N$ tokens at a time.

**Definition:** The **N-Grams of a text** are a list of tuples containing sliding windows of length $N$. A sliding window of size N moves over the text one token at a time, extracting continuous sequences of N words.

For instance, the trigrams in the sentence `'when I travel I become happy'` are:

```py
[('when', 'I', 'travel'), ('I', 'travel', 'I'), ('travel', 'I', 'become'), ('I', 'become', 'happy')]
```

<br>

#### Computing N-Gram Probabilities

In our trigram model above, we computed $P(\text{when I travel I am happy})$ as a product of conditional probabilities. These conditional probabilities are the result of **training** our N-Gram model on a training corpus.

To train an N-Gram model, we must compute a conditional probability for every $N$-token sequence in the corpus. For instance, when we are training a trigram model, for every 3-token sequence $w_1, w_2, w_3$, we must compute $P(w_3 | w_1, w_2)$. To do so, we use the definition of conditional probability:

$$P(w_3 | w_1, w_2) = \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)}$$

where $C(w_1, w_2, w_3)$ is the number of occurrences of the trigram sequence $w_1, w_2, w_3$ in the training corpus and $C(w_1, w_2)$ is the number of occurrences of the bigram sequence  $w_1, w_2$ in the training corpus.

In general, for any $N$, conditional probabilities are computed by dividing the counts of N-Grams by the counts of the (N-1)-Grams they follow. 


<br>

### The `NGramLM` Class 

1. **Creating an N-Gram Model**  
   - `NGramLM` takes in a **list of tokens**  and a number **N**.  
   - `N` must be **less than or equal to the total number of tokens**.  

2. **Generating N-Grams**  
   - The method `create_ngrams` extracts **overlapping N-word sequences** (tuples of length N) from the token list.  
   - These N-grams are then **used to train the model**.  

3. **Training the Model**  
   - The `train` method stores the N-gram data in a **DataFrame** with three columns:  
     - **`ngram`** → The actual N-gram (e.g., `("I", "become", "happy")`).  
     - **`n1gram`** → The **first (N-1) words** of the N-gram (e.g., `("I", "become")`).  
     - **`prob`** → The **probability** of the N-gram appearing in the text.  

4. **Storing Previous Models**  
   - The class keeps track of **(N-1)-gram models** inside `prev_mdl`.  
   - This helps calculate **probabilities for the first word(s) of a sentence**.  

The N-Gram LM consists of probabilities of the form

$$P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

which we estimate by  

$$\frac{C(w_{n-(N-1)}, w_{n-(N-2)}, \ldots, w_{n-1}, w_n)}{C(w_{n-(N-1)}, w_{n-(N-2)}, \ldots, w_{n-1})}$$

for every N-Gram that occurs in the corpus. 

Example:

```py
>>> corpus = 'hello, I am me, when I travel, I become happy, when I am at home, I am content.'
>>> tokens = tokenize(corpus)
>>> tokens
['\x02', 'hello', ',', 'I', 'am', 'me', ',', 'when', 'I', 'travel', ',', 'I', 'become', 'happy', ',', 'when', 'I', 'am', 'at', 'home', ',', 'I', 'am', 'content', '.', '\x03']
>>> model = NGrams(3, tokens)
```


To compute $P(\text{travel | when I})$, we must find the number of occurrences of `'when I travel'` in the training corpus, and divide it by the number of occurrences of `'I travel'` in the training corpus. `'when I travel'` occurred exactly once in the training corpus, while `'when I'` occurred twice, so,

$$P(\text{travel | when I}) = \frac{C(\text{when I travel})}{C(\text{when I})} = \frac{1}{2}$$

To store the conditional probabilities of all N-Grams, we will use a DataFrame with three columns as shown below:

|  | ngram                    | n1gram              |     prob |
|-------|-------------------------|--------------------|---------:|
| 0     | (',', 'I', 'become')     | (',', 'I')          | 0.333333 |
| 1     | ('I', 'am', 'at')        | ('I', 'am')         | 0.333333 |
| 2     | ('I', 'am', 'content')   | ('I', 'am')         | 0.333333 |
| 3     | ('I', 'am', 'me')        | ('I', 'am')         | 0.333333 |
| 4     | ('when', 'I', 'am')      | ('when', 'I')       | 0.5      |
| 5     | ('when', 'I', 'travel')  | ('when', 'I')       | 0.5      |
| 6     | (',', 'I', 'am')         | (',', 'I')          | 0.666667 |
| 7     | ('\x02', 'hello', ',')   | ('\x02', 'hello')   | 1        |
| 8     | (',', 'when', 'I')       | (',', 'when')       | 1        |
| 9   | ('I', 'become', 'happy') | ('I', 'become')     | 1        |
| 10    | ('I', 'travel', ',')     | ('I', 'travel')     | 1        |
| 11    | ('am', 'at', 'home')     | ('am', 'at')        | 1        |
| 12    | ('am', 'content', '.')   | ('am', 'content')   | 1        |
| 13    | ('am', 'me', ',')        | ('am', 'me')        | 1        |
| 14    | ('at', 'home', ',')      | ('at', 'home')      | 1        |
| 15    | ('become', 'happy', ',') | ('become', 'happy') | 1        |
| 16    | ('content', '.', '\x03') | ('content', '.')    | 1        |
| 17    | ('happy', ',', 'when')   | ('happy', ',')      | 1        |
| 18    | ('hello', ',', 'I')      | ('hello', ',')      | 1        |
| 19    | ('home', ',', 'I')       | ('home', ',')       | 1        |
| 20   | ('me', ',', 'when')      | ('me', ',')         | 1        |
| 21    | ('travel', ',', 'I')     | ('travel', ',')     | 1        |

- Each row contains the conditional probability for its corresponding trigram. For instance the first row in the dataframe shows that the probability of the trigram `(',', 'I', 'become')` conditioned on the bigram `(',', 'I')` is 1/3, as we computed above. 
- Note that many of the above conditional probabilities are equal to 1 because many trigrams and their corresponding bigrams each appeared only once, and $\frac{1}{1} = 1$. 


In [11]:
class NGramLM(object):
    
    def __init__(self, N, tokens):
        
        self.N = N

        # Create N-grams from tokens
        ngrams = self.create_ngrams(tokens)

        self.ngrams = ngrams
        # Train the model (calculate probabilities)
        self.mdl = self.train(ngrams)
        # Raise an exception if N is intially set to less than 2.
        if N < 2:
            raise Exception('N must be greater than 1')
        
        # Initialize lower-order models for first probabilities 
        elif N == 2:
            self.prev_mdl = UnigramLM(tokens)
        else:
            self.prev_mdl = NGramLM(N-1, tokens)

    def create_ngrams(self, tokens):
        # creates all possible ngrams from tokens
        lst = []
        for i in range(len(tokens)-(self.N - 1)):
            lst.append(tuple(tokens[i:self.N+i]))
        return lst
        
    def train(self, ngrams):

        # Count occurrences of N-grams and (N-1)-grams
        helper_func = lambda x: [tuple(i[:-1]) for i in x]
        ngram_counts = pd.Series(ngrams).value_counts()

        n1gram=helper_func(ngrams)
        n1gram_counts = pd.Series(n1gram).value_counts()

        # Compute probability of each N-gram
        probs = {}
        for i,j in ngram_counts.items():
            match = tuple(i[:-1]) 
            probs[i]= j / n1gram_counts[match]
        
        # Create a probability distribution DataFrame 
        df =pd.DataFrame({"ngram": probs.keys(), "prob": probs.values()})
        df['n1gram'] = df['ngram'].apply(lambda i: tuple(i[:-1]) )
        return df.iloc[:,[0,2,1]].drop_duplicates().sort_values(['prob','ngram','n1gram'])
    
    def probability(self, words):

        # Generate possible N-grams from input words
        my_ngrams = []
        for i in range(len(words),0,-1):
            if len(words[:i]) > self.N:
                my_ngrams.append(tuple(words[:i][-self.N:]))
            else:
                my_ngrams.append(tuple(words[:i]))
        final_prob= 1
        obj = self
        for i in my_ngrams:
            n = len(i)

            # Navigate back to the appropriate N-gram model if needed
            while obj.N!=n:
                obj = obj.prev_mdl
                if isinstance(obj,UnigramLM):
                    break
            if n>1:
                df = obj.mdl
                df = df[df['ngram']==i]
                if df.shape[0]==0:
                    return 0 # If the N-gram is not found, return zero probability
                final_prob *= df['prob'].iloc[0]
                
            else:
                final_prob *= obj.probability(i)
            
        return final_prob
       
    def sample(self, M):

        out = ['\x02'] # Start symbol
        for i in range(M-1):
            beg = out[-(self.N - 1):] # Get the last (N-1) words as context
            n_needed = len(beg)+1
            obj = self
            # Navigate back to the appropriate N-gram model if needed
            while obj.N!=n_needed:
                    obj = obj.prev_mdl
                    if isinstance(obj,UnigramLM):
                        break
            df = obj.mdl
            df = df[df['n1gram']==tuple(beg)]
            if df.shape[0]==0:
                out.append("\x03") # End symbol if no match found
                continue
             # Sample next word based on probabilities
            props = df.set_index('ngram')["prob"]
            next_word = np.random.choice(props.index.to_numpy(),1,p=props.to_numpy())[0][-1]
            out.append(next_word)
        out.append("\x03") # End symbol
        return ' '.join(out)


Below we try our n-gram model using the same Frankenstein tokenized text/corpus.

In [12]:

ngram = NGramLM(3, Frankenstein_tokenized)

# all possible 3-ngrams
ngram.ngrams[:20] # first 20 3-ngrams

[('\x02', 'Frankenstein', ';'),
 ('Frankenstein', ';', '\x03'),
 (';', '\x03', '\x02'),
 ('\x03', '\x02', 'or'),
 ('\x02', 'or', ','),
 ('or', ',', 'the'),
 (',', 'the', 'Modern'),
 ('the', 'Modern', 'Prometheus'),
 ('Modern', 'Prometheus', '\x03'),
 ('Prometheus', '\x03', '\x02'),
 ('\x03', '\x02', 'by'),
 ('\x02', 'by', 'Mary'),
 ('by', 'Mary', 'Wollstonecraft'),
 ('Mary', 'Wollstonecraft', '('),
 ('Wollstonecraft', '(', 'Godwin'),
 ('(', 'Godwin', ')'),
 ('Godwin', ')', 'Shelley'),
 (')', 'Shelley', '\x03'),
 ('Shelley', '\x03', '\x02'),
 ('\x03', '\x02', 'CONTENTS')]

In [78]:
# ngram probability distribution when n=3
ngram.mdl

,ngram,n1gram,prob
30847,"(,, and, Daniel)","(,, and)",0.001058
16345,"(,, and, De)","(,, and)",0.001058
24203,"(,, and, Ernest)","(,, and)",0.001058
10497,"(,, and, Greenwich)","(,, and)",0.001058
36911,"(,, and, Henry)","(,, and)",0.001058
...,...,...,...
4771,"(”, but, I)","(”, but)",1.000000
27416,"(”, interrupted, the)","(”, interrupted)",1.000000
1796,"(”, she, said)","(”, she)",1.000000
65315,"(”, were, not)","(”, were)",1.000000


In [79]:
# probability of greatest practical
ngram.probability(('greatest','practical'))

1.14195662848725e-05

In [64]:
# Generates a text of 500 words through the dataframe of probability distributions. 
print(ngram.sample(500))

 It was on a bed , hardly conscious of what they said , “ will you have yourself described to be informed of the fire of love to bestow .   But I thought of the accumulated ice , among merchants and seamen . Yet at the bottom of the grave - worms crawling in the least interest me , and the spirits who assist my vengeance in his boat , with a feeble voice , “ how kind , how very ill , all capacity of bestowing animation upon lifeless matter , ” said I was dependent on none and related to none . ‘ Who are you ? I have no one was too potent , and but one vast hand was already dusk before we thought of my early years . You were thrown into prison the very stars themselves being witnesses and arranging my chemical apparatus .   I mentioned before , which were at first ; and the rest necessary for my guest which this tale of misery , according as they roared and dashed at my bedside ; its astounding horror would be pursuit ? Who was I really as mad as the peasants ; and you will , destr